In [1]:
import requests
import json 
import pandas as pd 
import time
import pycountry
from json_flatten import flatten
from sqlalchemy import create_engine
from datetime import datetime
import pytz
from timezonefinder import TimezoneFinder

In [3]:
OW_api_key = '1bb3d74b03e16c97feaddad9963cd29d'
Nin_api_key = 'moIsb6GGrVPEVN0UzWFQhA==HLX28kq15hgK6EcC'


Finalise List of Cities to use

In [28]:
pop_ranking = pd.read_csv(rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Raw\world-city-listing-table.csv')


pop_ranking = pop_ranking[pop_ranking['population']>4000000]

def country_to_iso_alpha2(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_2
    except LookupError:
        return None  
pop_ranking['country_iso'] = pop_ranking['country'].apply(country_to_iso_alpha2)

countries = set(pop_ranking["country"])
cities = pop_ranking[['city','country_iso']]



City Lon and Lat


In [ ]:
cities_coord = []
delay = 2

#cities = cities[1:2]

for index, row in cities.iterrows():
    city = row['city']
    country = row['country_iso']

    api_url = f'https://api.api-ninjas.com/v1/city?name={city}&country={country}&min_population=4500000'
    response = requests.get(api_url , headers={'X-Api-Key': Nin_api_key})
    if response.status_code == requests.codes.ok:
        
        cities_coord.append(response.text)
        time.sleep(delay)
    else:
        print("Error:", response.status_code, response.text)
print(cities_coord)


In [120]:

flattened_list = []

for json_str in cities_coord:

    list_of_dicts = json.loads(json_str)
    
    if isinstance(list_of_dicts, list):
        flattened_list.extend(list_of_dicts)
    else:
        print(f"Unexpected data format: {list_of_dicts}")


print(flattened_list)
file_path = rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Raw\cities_geodata_list.json'

with open(file_path, 'w') as f:
    json.dump(flattened_list, f, indent=4)

[{'name': 'Tokyo', 'latitude': 35.6897, 'longitude': 139.692, 'country': 'JP', 'population': 37977000, 'is_capital': True}, {'name': 'Delhi', 'latitude': 28.66, 'longitude': 77.23, 'country': 'IN', 'population': 29617000, 'is_capital': False}, {'name': 'Shanghai', 'latitude': 31.1667, 'longitude': 121.467, 'country': 'CN', 'population': 22120000, 'is_capital': False}, {'name': 'Dhaka', 'latitude': 23.7161, 'longitude': 90.3961, 'country': 'BD', 'population': 15443000, 'is_capital': True}, {'name': 'Sao Paulo', 'latitude': -23.5504, 'longitude': -46.6339, 'country': 'BR', 'population': 22046000, 'is_capital': False}, {'name': 'Cairo', 'latitude': 30.0561, 'longitude': 31.2394, 'country': 'EG', 'population': 19372000, 'is_capital': True}, {'name': 'Mexico City', 'latitude': 19.4333, 'longitude': -99.1333, 'country': 'MX', 'population': 20996000, 'is_capital': True}, {'name': 'Beijing', 'latitude': 39.905, 'longitude': 116.391, 'country': 'CN', 'population': 19433000, 'is_capital': True},

OpenWeather API

In [2]:
file_path = rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Raw\cities_geodata_list.json'


data_to_csv = pd.read_json(file_path)
data_to_csv.rename(columns={'name': 'city_name'}, inplace=True)
data_to_csv['city_id'] = pd.factorize(data_to_csv['city_name'])[0] + 1

csv_save_path = rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Final\Final_city_data.csv'
json_save_path = rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Final\Final_city_data.json'
data_to_csv.to_csv(csv_save_path, index= False)

city_data_json = data_to_csv.to_json(orient='records')
city_data = json.loads(city_data_json)
with open(json_save_path, 'w') as f:
    json.dump(city_data, f, indent=4)



OSError: Cannot save file into a non-existent directory: 'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Final'

In [4]:

OW_weather_data = []
OW_pollution_data = []
delay = 2

json_save_path = rf'D:\Datasets\Weather and Pollution\Data\Final\Final_city_data.json'
with open(json_save_path) as f:
    city_data = json.load(f)

city_data = city_data[:2]


for city in city_data:

     #Load the longitude and latitude data for each city
     lat = city['latitude']
     lon = city['longitude']
     city_id = city['city_id']

     #Fetch the current weather data 
     OW_weather_url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OW_api_key}'
     response_weather = requests.get(OW_weather_url)
     weather_data = response_weather.json()
     weather_data['latitude'] = lat
     weather_data['longitude'] = lon
     weather_data['city_id'] = city_id
     OW_weather_data.append(weather_data)
     time.sleep(delay)

     #Fetch the current air quality data
     OW_pollution_url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={OW_api_key}'
     response_pollution = requests.get(OW_pollution_url)
     pollution_data = response_pollution.json()
     pollution_data['latitude'] = lat
     pollution_data['longitude'] = lon
     pollution_data['city_id'] = city_id
     OW_pollution_data.append(pollution_data)
     time.sleep(delay)


#Save each file 
OW_raw_weather_filepath = rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Raw\OW_raw_weather_data.json'
with open(OW_raw_weather_filepath, 'w') as f:
    json.dump(OW_weather_data, f, indent=4)

OW_raw_pollution_filepath = rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Raw\OW_raw_pollution_data.json'
with open(OW_raw_pollution_filepath, 'w') as f:
    json.dump(OW_pollution_data, f, indent=4)

In [5]:

def rename_and_convert_columns(df, column_map):
    """
    Renames columns in a DataFrame and converts them to the specified data types.

    Args:
        df (pd.DataFrame): The DataFrame to process.
        column_map (dict): A dictionary where keys are original column names and values are tuples of 
                           the new column name and data type (e.g., ('new_name', 'dtype')).

    Returns:
        pd.DataFrame: The modified DataFrame with renamed columns and converted data types.
    """
    for original_col, (new_col, dtype) in column_map.items():
        if original_col in df.columns:
            df.rename(columns={original_col: new_col}, inplace=True)
            
            # Convert to appropriate data type
            if dtype == 'float':
                df[new_col] = pd.to_numeric(df[new_col], errors='coerce')
            elif dtype == 'int':
                df[new_col] = pd.to_numeric(df[new_col], errors='coerce', downcast='integer')
            elif dtype == 'datetime':
                # Explicitly cast to numeric before converting to datetime
                df[new_col] = pd.to_datetime(pd.to_numeric(df[new_col], errors='coerce'), unit='s', errors='coerce')
                df[new_col] = df[new_col].dt.round('H')
    return df



In [6]:

def calculate_local_time(datetime_str, lat, lon):
    """
    Calculates the local time based on latitude, longitude, and UK local datetime.

    Parameters:
    datetime_str (str): The datetime in UK local time as a string.
    lat (float): Latitude of the location.
    lon (float): Longitude of the location.

    Returns:
    pd.Timestamp: The local time as a Pandas Timestamp object.
    """
  
    # Parse the UK local datetime string into a datetime object
    uk_datetime = pd.to_datetime(datetime_str)

    # Initialize TimezoneFinder
    tf = TimezoneFinder()

    # Find timezone based on latitude and longitude
    timezone_str = tf.timezone_at(lng=lon, lat=lat)

    # Get the timezone
    timezone = pytz.timezone(timezone_str)
    # Convert the UK local time to local time in the identified timezone
    local_time = pd.Timestamp(uk_datetime).tz_localize(pytz.timezone('Europe/London')).astimezone(timezone)
    return local_time.tz_localize(None)
    


In [7]:
OW_raw_weather_filepath = rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Raw\OW_raw_weather_data.json'

with open(OW_raw_weather_filepath) as f:
    raw_weather_data = json.load(f)
flattened_weather_data = [flatten(item) for item in raw_weather_data]

# Convert to DataFrame
weather_data_df = pd.DataFrame(flattened_weather_data)

# Define a mapping for column renaming and data type conversions
weather_column_map = {
    'city_id$int': ('city_id', 'int'),
    'longitude$float': ('longitude', 'float'),
    'latitude$float': ('latitude', 'float'),
    'weather.[0].id$int': ('weather_id', 'int'),
    'weather.[0].main': ('weather_main', 'str'),
    'weather.[0].description': ('weather_description', 'str'),
    'weather.[0].icon': ('weather_icon', 'str'),
    'main.temp$float': ('temperature', 'float'),
    'main.feels_like$float': ('feels_like', 'float'),
    'main.temp_min$float': ('temp_min', 'float'),
    'main.temp_max$float': ('temp_max', 'float'),
    'main.pressure$int': ('pressure', 'int'),
    'main.humidity$int': ('humidity', 'int'),
    'main.sea_level$int': ('sea_level', 'int'),
    'main.grnd_level$int': ('grnd_level', 'int'),
    'visibility$int': ('visibility', 'int'),
    'wind.speed$float': ('wind_speed', 'float'),
    'wind.deg$int': ('wind_deg', 'int'),
    'clouds.all$int': ('clouds_all', 'int'),
    'dt$int': ('date_time', 'datetime'),
    'sys.type$int': ('sys_type', 'int'),
    'sys.id$int': ('sys_id', 'int'),
    'sys.country': ('sys_country', 'str'),
    'sys.sunrise$int': ('sunrise', 'datetime'),
    'sys.sunset$int': ('sunset', 'datetime'),
    'timezone$int': ('timezone', 'int'),
    'id$int': ('id', 'int'),
    'name': ('name', 'str'),
    'cod$int': ('cod', 'int'),
    'wind.gust$float': ('wind_gust', 'float'),
    'rain.1h$float': ('rain_1h', 'float')
}

weather_data_df = rename_and_convert_columns(weather_data_df, weather_column_map)
weather_data_df['local_time'] = weather_data_df.apply(
    lambda row: calculate_local_time(row['date_time'], row['latitude'], row['longitude']),
    axis=1
)


# Define the columns to keep
required_columns = [
    'city_id',  'date_time', 'local_time','temperature', 'feels_like', 'temp_min', 'temp_max',
    'pressure', 'humidity', 'visibility', 'wind_speed', 'wind_deg', 'clouds_all',
    'weather_main', 'weather_description', 'weather_icon', 'sunrise', 'sunset'
]


# Filter DataFrame to keep only the required columns
weather_data_df = weather_data_df[required_columns]

# Save the DataFrame to a CSV file
weather_data_df.to_csv('weather_data_df.csv', index=False)

print(weather_data_df.dtypes)


C:\Users\gurpr\AppData\Local\Temp\ipykernel_12032\1439812154.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df[new_col] = df[new_col].dt.round('H')
C:\Users\gurpr\AppData\Local\Temp\ipykernel_12032\1439812154.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df[new_col] = df[new_col].dt.round('H')
C:\Users\gurpr\AppData\Local\Temp\ipykernel_12032\1439812154.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df[new_col] = df[new_col].dt.round('H')


city_id                          int8
date_time              datetime64[ns]
local_time             datetime64[ns]
temperature                   float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
pressure                        int16
humidity                         int8
visibility                      int16
wind_speed                    float64
wind_deg                        int16
clouds_all                       int8
weather_main                   object
weather_description            object
weather_icon                   object
sunrise                datetime64[ns]
sunset                 datetime64[ns]
dtype: object


In [2]:

def convert_uk_local_to_city_local(lat, lon, uk_local_time):
    """
    Convert UK local time to the local time of a city based on latitude and longitude.
    
    Args:
    - lat (float): Latitude of the city.
    - lon (float): Longitude of the city.
    - uk_local_time (datetime): A datetime object in UK local time (GMT/BST).
    
    Returns:
    - local_time (datetime): The datetime converted to local time of the city.
    """
    # Define the timezone for the UK and Convert UK local time to UTC
    uk_timezone = pytz.timezone('Europe/London')
    utc_time = uk_local_time.astimezone(pytz.utc)

    # Determine the timezone based on latitude and longitude
    tz_finder = tzwhere.tzwhere()
    timezone_str = tz_finder.tzNameAt(lat, lon)

    # Convert UTC time to local time
    timezone = pytz.timezone(timezone_str)
    local_time = utc_time.astimezone(timezone)
    return local_time


In [8]:
OW_raw_pollution_filepath = rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data\Raw\OW_raw_pollution_data.json'
with open(OW_raw_pollution_filepath) as f:
    raw_pollution_data = json.load(f)
flattened_pollution_data = [flatten(item) for item in raw_pollution_data]

pollution_data_df = pd.DataFrame(flattened_pollution_data)

# Define a mapping for column renaming and data type conversions
pollution_column_map = {
    'city_id$int': ('city_id', 'int'),
    'list.[0].dt$int': ('date_time', 'datetime'),
    'longitude$float': ('longitude', 'float'),
    'latitude$float': ('latitude', 'float'),
    'list.[0].main.aqi$int': ('aqi', 'int'),
    'list.[0].components.co$float': ('co', 'float'),
    'list.[0].components.no$int': ('no', 'floT'),
    'list.[0].components.no2$float': ('no2', 'float'),
    'list.[0].components.o3$float': ('o3', 'float'),
    'list.[0].components.so2$float': ('so2', 'float'),
    'list.[0].components.pm2_5$float': ('pm2_5', 'float'),
    'list.[0].components.pm10$float': ('pm10', 'float'),
    'list.[0].components.nh3$float': ('nh3', 'float')
}

# Rename columns and convert data types
pollution_data_df = rename_and_convert_columns(pollution_data_df, pollution_column_map)

# Define the columns to keep
required_pollution_columns = [col for col in pollution_column_map.values() if col[0] in pollution_data_df.columns]

# Filter DataFrame to keep only the required columns
pollution_data_df = pollution_data_df[[col[0] for col in required_pollution_columns]]
pollution_data_df['local_time'] = pollution_data_df.apply(
    lambda row: calculate_local_time(row['date_time'], row['latitude'], row['longitude']),
    axis=1
)

pollution_data_df=pollution_data_df.drop(columns=['latitude','longitude'])


# Save the DataFrame to a CSV file
pollution_data_df.to_csv('pollution_data_df.csv', index=False)

print(pollution_data_df.dtypes)


C:\Users\gurpr\AppData\Local\Temp\ipykernel_12032\1439812154.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df[new_col] = df[new_col].dt.round('H')


city_id                 int8
date_time     datetime64[ns]
aqi                     int8
co                   float64
no2                  float64
o3                   float64
so2                  float64
pm2_5                float64
pm10                 float64
nh3                  float64
local_time    datetime64[ns]
dtype: object


In [3]:
#Load all credentials 
with open(rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data_ETL\config.json') as config_file:
    config = json.load(config_file)


In [ ]:

OW_api_key = config['OW_api_key']
db_name = config['db_name']
db_user = config['user']
db_pass = config['password']

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}\
@localhost/{db_name}')

conn = engine.connect()

timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Define paths for saving CSV files with timestamps
weather_csv_path = rf'D:\Datasets\Weather and Pollution\Data\Failed to Upload data\Weather\weather_data_{timestamp}.csv'
pollution_csv_path = rf'D:\Datasets\Weather and Pollution\Data\Failed to Upload data\Pollution\pollution_data_{timestamp}.csv'

try:
    weather_data_df.to_sql('weather', con=conn, if_exists='append', index=False)

except Exception as e:
    weather_data_df.to_csv(weather_csv_path, index=False)

try:
    pollution_data_df.to_sql('pollution', con=conn, if_exists='append', index=False)

except Exception as e:
    pollution_data_df.to_csv(pollution_csv_path, index=False)
    


In [5]:

# Database connection details
db_name = config['db_name']
db_user = config['user']
db_pass = config['password']
db_host = config['host']
db_port = config['port']
# Create the SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}\
@{db_host}:{db_port}/{db_name}')


# Function to test the connection
def test_connection():
    try:
        # Establish a connection using a context manager
        with engine.connect() as connection:
            print("Connection successful!")
            # Execute a simple query to verify connection
            result = connection.execute("SELECT 1")
            print("Query result:", result.fetchone())
    except Exception as e:
        print("Connection failed:", e)

# Test the connection
test_connection()

Connection failed: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [ ]:

# Define paths for saving CSV files with timestamps
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
weather_csv_path = rf'D:\Datasets\Weather and Pollution\Data\Failed to Upload data\Weather\weather_data_{timestamp}.csv'
pollution_csv_path = rf'D:\Datasets\Weather and Pollution\Data\Failed to Upload data\Pollution\pollution_data_{timestamp}.csv'

try:
    # Use the engine directly in to_sql
    weather_data_df.to_sql('weather', con=engine, if_exists='append', index=False)
    print("Successfully uploaded weather data.")
except Exception as e:
    print(f"Failed to upload weather data: {e}")
    weather_data_df.to_csv(weather_csv_path, index=False)

try:
    # Use the engine directly in to_sql
    pollution_data_df.to_sql('pollution', con=engine, if_exists='append', index=False)
    print("Successfully uploaded pollution data.")
except Exception as e:
    print(f"Failed to upload pollution data: {e}")
    pollution_data_df.to_csv(pollution_csv_path, index=False)
